<a href="https://colab.research.google.com/github/akshaypt7/amazon_sales_prediction/blob/main/trying_embedding_for_amazon_sales_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 720 kB 5.3 MB/s 
     |████████████████████████████████| 1.2 MB 34.7 MB/s 
     |████████████████████████████████| 46 kB 3.2 MB/s 
     |████████████████████████████████| 188 kB 53.9 MB/s 
     |████████████████████████████████| 53 kB 1.7 MB/s 
     |████████████████████████████████| 51 kB 193 kB/s 
Mounted at /content/gdrive


In [2]:
#hide
from fastbook import *
# from kaggle import api
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
# from dtreeviz.trees import *
from IPython.display import Image, display_svg, SVG

pd.options.display.max_rows = 20
pd.options.display.max_columns = 8

In [3]:
from sklearn.model_selection import ParameterGrid
from keras.models import Sequential
from keras.models import Model as KerasModel
from keras.layers import Input, Dense, Activation, Reshape
from keras.layers import Concatenate
from keras.layers import Dense, Flatten, Dropout, Input, Embedding, Reshape
from keras.layers import concatenate
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
import pickle
import csv
from datetime import datetime
from sklearn import preprocessing
from keras.callbacks import ModelCheckpoint
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
%matplotlib inline

from sklearn.model_selection import train_test_split
import math
import pandas as pd

In [37]:
infile = open('/content/gdrive/MyDrive/bluebook/df_main.pkl','rb')
df_main = pickle.load(infile)

In [38]:
df_main = add_datepart(df_main,'date')

In [7]:
cat = ['Asin',
 'Is_month_end',
 'Is_month_start',
 'Is_quarter_end',
 'Is_quarter_start',
 'Is_year_end',
 'Is_year_start',
 'hour','Month','Week', 'Day','Dayofweek','Dayofyear']

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

In [39]:
enc = OrdinalEncoder()
enc.fit(df_main[cat])
df_main[cat]= enc.transform(df_main[cat])

In [40]:
condition = (df_main.Year < 2020) | (df_main.Month < 8)
train_idx = np.where(condition)[0]
valid_idx = np.where(~condition)[0]

In [17]:
# X_train, X_test, y_train, y_test = train_test_split(df_main.drop(['Quantity'],axis=1), df_main['Quantity'], test_size=0.1, random_state=42)

In [41]:
X_train = df_main.drop(['Quantity'],axis=1).iloc[train_idx]
y_train = df_main['Quantity']

X_valid = df_main.drop(['Quantity'],axis=1).iloc[valid_idx]
y_valid = df_main['Quantity']

In [42]:
X_train.shape, X_valid.shape

((8423, 18), (2523, 18))

In [22]:
# embedding for ASiN
n_unique_asins = df_main.Asin.nunique()
n_dim_asins = int(math.sqrt(n_unique_asins))


input_asin = Input(shape=(1,))
output_asin = Embedding(input_dim = n_unique_asins, output_dim= n_dim_asins, name='asin')(input_asin)
output_asin = Reshape(target_shape=(n_dim_asins,))(output_asin)

In [23]:
# embedding for hour
n_unique_hour = df_main.hour.nunique()
n_dim_hours = int(math.sqrt(n_unique_hour))

input_hour = Input(shape=(1,))
output_hour = Embedding(input_dim= n_unique_hour, output_dim=n_dim_hours,name='hours')(input_hour)
output_hour = Reshape( target_shape= (n_dim_hours,))(output_hour)

In [24]:
# day_of week
n_unique_day_of_week = df_main['Dayofweek'].nunique()
n_dim_dayofweek = 3

input_dayofweek = Input(shape=(1,))
output_dayofweek = Embedding(input_dim= n_unique_day_of_week, output_dim=n_dim_dayofweek,name='dayofweek')(input_dayofweek)
output_dayofweek = Reshape( target_shape= (n_dim_dayofweek,))(output_dayofweek)

In [25]:
# Month
n_unique_month = df_main['Month'].nunique()
n_dim_month = 7

input_month = Input(shape=(1,))
output_month = Embedding(input_dim= n_unique_month, output_dim=n_dim_month,name='month')(input_month)
output_month = Reshape( target_shape= (n_dim_month,))(output_month)

In [26]:
# Day (1-31)
n_unique_day = df_main['Day'].nunique()
n_dim_day = int(math.sqrt(n_unique_day))

input_day = Input(shape=(1,))
output_day= Embedding(input_dim= n_unique_day, output_dim=n_dim_day,name='day')(input_day)
output_day = Reshape( target_shape= (n_dim_day,))(output_day)

In [27]:
# Week (1-53)

n_unique_week = df_main['Week'].nunique()
n_dim_week = 10

input_week = Input(shape=(1,))
output_week= Embedding(input_dim= n_unique_week, output_dim=n_dim_week,name='week')(input_week)
output_week = Reshape( target_shape= (n_dim_week,))(output_week)


In [28]:
input_layers = [input_asin, input_hour, input_dayofweek, input_month, input_day, input_week ]
output_layers = [output_asin, output_hour, output_dayofweek, output_month, output_day, output_week ] 
model = Concatenate()(output_layers)

In [29]:

model = Dense(1000, kernel_initializer='uniform')(model)
model = Activation('relu')(model)

model = Dense(500, kernel_initializer='uniform')(model)
model = Activation('relu')(model)

model = Dense(1)(model)
model = Activation('sigmoid')(model)

In [30]:
modelNN_emb = KerasModel(inputs=input_layers, outputs=model)
modelNN_emb.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 1)]          0                                            
______________________________________________________________________________________________

In [31]:


from tensorflow.keras.optimizers import Adam
opt =  Adam(lr=0.05)
modelNN_emb.compile(loss='mae', optimizer=opt, metrics=['mae'])



/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [32]:
X_train.columns

Index(['Asin', 'Item Promo Discount', 'price', 'hour', 'Invoice Amount',
       'Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
       'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start',
       'Is_year_end', 'Is_year_start', 'Elapsed'],
      dtype='object')

In [34]:
def split_features(X):
    X_list = []

    asin = X[:, [0]].reshape(-1,1)#
    X_list.append(asin)

    hour = X[:, [3]].reshape(-1,1)#
    X_list.append(hour)

    dayofweek = X[:, [9]].reshape(-1,1)
    X_list.append(dayofweek)

    month = X[:, [6]].reshape(-1,1)
    X_list.append(month)

    day = X[:, [8]].reshape(-1,1)
    X_list.append(day)

    week = X[:, [7]].reshape(-1,1)
    X_list.append(week)

    return X_list

In [35]:
def preprocessing(X):
    X_list = split_features(X)
    return X_list

In [36]:


modelNN_emb.checkpointer = ModelCheckpoint(filepath="best_model_weights.hdf5", verbose=1, save_best_only=True)
modelNN_emb.fit(preprocessing(X_train.values), y_train, epochs = 10, batch_size = 128, validation_data= (preprocessing(X_test.values),y_test), callbacks=[modelNN_emb.checkpointer])



Epoch 1/10
77/77 [==============================] - 20s 29ms/step - loss: 0.2457 - mae: 0.2457 - val_loss: 0.1845 - val_mae: 0.1845

Epoch 00001: val_loss improved from inf to 0.18447, saving model to best_model_weights.hdf5
Epoch 2/10
77/77 [==============================] - 1s 17ms/step - loss: 0.2241 - mae: 0.2241 - val_loss: 0.1845 - val_mae: 0.1845

Epoch 00002: val_loss did not improve from 0.18447
Epoch 3/10
77/77 [==============================] - 1s 17ms/step - loss: 0.2223 - mae: 0.2223 - val_loss: 0.1845 - val_mae: 0.1845

Epoch 00003: val_loss did not improve from 0.18447
Epoch 4/10
77/77 [==============================] - 1s 17ms/step - loss: 0.2203 - mae: 0.2203 - val_loss: 0.1845 - val_mae: 0.1845

Epoch 00004: val_loss did not improve from 0.18447
Epoch 5/10
77/77 [==============================] - 1s 17ms/step - loss: 0.2125 - mae: 0.2125 - val_loss: 0.1845 - val_mae: 0.1845

Epoch 00005: val_loss did not improve from 0.18447
Epoch 6/10
77/77 [=========================

In [ ]:
store_embedding = modelNN_emb.get_layer('store_embedding').get_weights()[0]
dow_embedding = modelNN_emb.get_layer('dow_embedding').get_weights()[0]
year_embedding = modelNN_emb.get_layer('year_embedding').get_weights()[0]
month_embedding = modelNN_emb.get_layer('month_embedding').get_weights()[0]
day_embedding = modelNN_emb.get_layer('day_embedding').get_weights()[0]
german_states_embedding = modelNN_emb.get_layer('state_embedding').get_weights()[0]
with open('/content/gdrive/MyDrive/bluebook', 'wb') as f:
    pickle.dump([store_embedding, dow_embedding, year_embedding,
                  month_embedding, day_embedding, german_states_embedding], f, -1)